"The Adult dataset is from the Census Bureau and the task is to predict whether a given adult makes more than $50,000 a year based attributes such as education, hours of work per week, etc.."
The dataset was drawn from the 1994 United States Census Bureau data and is credited to Ronny Kohavi and Barry Becker. 


*   Target field: Income (The income is divide into two classes: <=50K and >50K )
*   A certain number of attributes (variables): These are the demographics and other features to describe a person

For more information regarding the dataset see [Adult income](https://archive.ics.uci.edu/ml/datasets/adult)


Code for importing the required packages (is given)

In [2]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics

from matplotlib import pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import Input, Model, layers, losses, optimizers, callbacks

2022-11-08 23:54:55.083994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


The Url and column names of the dataset are given. Read this dataset into a pandas dataframe and store it as a csv file. 

In [136]:
# load the dataset
url_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

column_names = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num','marital-status',
                'occupation', 'relationship', 'race', 'gender','capital-gain', 'capital-loss', 
                'hours-per-week', 'native-country','income']


df = pd.read_csv(url_data, names=column_names)
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Provide a **description** of the dataset in pandas

In [137]:
# Describe the dataframe

df.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


Find out and print the unique values per column 

In [138]:
# Print out unique values per column

for col, data in df.items() :
    out = pd.unique(data)
    print(col, len(out) ,'\n' ,out, '\n')

age 73 
 [39 50 38 53 28 37 49 52 31 42 30 23 32 40 34 25 43 54 35 59 56 19 20 45
 22 48 21 24 57 44 41 29 18 47 46 36 79 27 67 33 76 17 55 61 70 64 71 68
 66 51 58 26 60 90 75 65 77 62 63 80 72 74 69 73 81 78 88 82 83 84 85 86
 87] 

workclass 9 
 [' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked'] 

fnlwgt 21648 
 [ 77516  83311 215646 ...  34066  84661 257302] 

education 16 
 [' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th'] 

educational-num 16 
 [13  9  7 14  5 10 12 11  4 16 15  3  6  2  1  8] 

marital-status 7 
 [' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed'] 

occupation 15 
 [' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' '

Compare the columns 'education' and 'educational-num', drop the column 'education'.

In [139]:
df['education'].compare(df['educational-num'])

df = df.drop('education', axis=1)

Remove whitespaces at the beginning and end of all attributes of type String and drop the rows containing '?'

In [141]:
# Drop rows containing '?'
df = df.applymap(lambda x : x.strip() if isinstance(x, str) else x)
    
df['native-country'].unique()

#df['native-country'].str.contains('?', regex=False)
mask = (df == '?').any(axis=1)

df = df.drop(df[mask].index)


Provide a **description** of your dataframe again. Did the number of rows reduce?

In [131]:
# A description of your dataframe
df.describe()
# We dropped around 20.000 rows

30162

Make a new column with 'age-groups' and drop 'age'


In [142]:
# Make new column with 'age-groups' and drop 'age'
age_bins = [0, 18, 25, 35, 45, 55, 65, 100]

Print two lists containing:


*   Only numerical columns
*   Only Categorical columns




In [ ]:
# get new columns, cat and num cols


Normalize all numerical columns

In [ ]:
# Normalize all numerical columns


Plot the column distributions (one plot for each column)
* Hint: lookup [Seaborn histplot](https://seaborn.pydata.org/generated/seaborn.histplot.html) for numerical columns
* Hint: lookup [Seaborn countplot](https://seaborn.pydata.org/generated/seaborn.countplot.html) for categorical columns

In [ ]:
# Plot distributions of columns


Label encode your categorical columns and print label encodings for each variable

In [ ]:
# Label encoding of categorical columns
from sklearn.preprocessing import LabelEncoder


# Print label encodings for each variable


Plot heatmap of cross correlations

Hint: lookup [Seaborn heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) and [pairwise correlation of columns](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)

Compute the correlation of each column with the target column 'income'

In [ ]:
# Heatmap of cross correlations


# correlation of columns with target 'income' (print them)


Provide a pairplot of columns where the correlation with 'income' is higher than 0.1:


* Hint: See [Plot pairwise relationships in a dataset](https://seaborn.pydata.org/generated/seaborn.pairplot.html)
* Hint: Subsample the data for plotting (otherwise it will take forever to render), you can use 1000 random samples
* Hint: Set hue='income' in sns.pairplot




In [ ]:
# Pairplot of columns correlated with income


Do a [one-hot encoding](https://en.wikipedia.org/wiki/One-hot) of all categorical columns.  
Hint: See [pandas.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html).
* Print the first five rows of your new dataframe.

In [ ]:
# One-hot encoding

Split the data into trainset, valset (1000 samples) and testset (10000 samples). Print the shape of your training, validation and test data. 

In [ ]:
# Split the data into trainset, valset (1000 samples) and testset (10000 samples)


Write a function that splits the dataframe into X,Y, where Y can be any of the (possibly one-hot encoded) columns 

In [ ]:
# Write a function thats splits the dataframe into X,Y, where Y can be any column
def split_xy(df, colname):

    return X, Y

# X, Y = split_xy(df_train, 'income') // 
# print(X.shape, Y.shape) // 

(19169, 96) (19169, 2)


Write and train a linear classifier with tensorflow keras to predict 'income'


*   Loss: CategoricalCrossentropy





In [ ]:
# Train a linear classifier to predict 'income'


Print the classification report and confusion matrix 

In [ ]:
# Classification report + confusion matrix

* Now try some other models such as K-nearest neighbors, Random Forest classifier, and Support Vector Machine (code for importing the classifiers is given below)
* Print the classification report and confusion matrix of each classifier 

In [ ]:
# Try out non-linear models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC



Train a neural network for 50 epochs with:
* An input layer
* Two dense layers, where between the two dense layers is a dropout layer
* Input -> Dense -> Dropout-> Dense

* Loss: CategoricalCrossentropy
* Metrics: 'accuracy'

In [ ]:
# Try out non-linear NN


Print the classification report and confusion matrix 

Did the non-linear classifiers improve over the linear classifier? Interprete the results.